In [1]:
from jsonobject import *
import numpy as np 
import random

In [2]:
random.seed(2021)

# Classes

In [3]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [4]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)

In [5]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="'valid'")
    stride = IntegerProperty(default=None)

In [6]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [7]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

# Test

In [8]:
conv1 = Convolution(kernel=5, padding="'valid'", stride=2, nb_filter=6, fct_activation="'tanh'")
conv1.to_json()

{'kernel': 5,
 'padding': "'valid'",
 'stride': 2,
 'nb_filter': 6,
 'fct_activation': "'tanh'"}

In [9]:
inputL = InputLayer(shape=[28,28,1], kernel=5, padding="'valid'", stride=2, nb_filter=6, fct_activation="'tanh'")
inputL.to_json()

{'shape': [28, 28, 1],
 'kernel': 5,
 'padding': "'valid'",
 'stride': 2,
 'nb_filter': 6,
 'fct_activation': "'tanh'"}

In [10]:
pool1 = Pooling(op = "avg")
pool1.to_json()

{'op': 'avg', 'kernel': 2, 'padding': "'valid'", 'stride': None}

# Values

In [11]:
layers_name = [Convolution, InputLayer, Pooling, Flatten, Dense]

# Table of hyperparameter value
kernel_value = [1, 2, 3, 5, 7]
stride_value = [1, 2, 3, 4, 5]
padding_value = ["valid", "same"]
nb_filter_value = [6, 16, 64, 128, 256, 512, 1024, 2048]
fct_activation_value = ["tanh", "sigmoid", "relu", "selu"]
nb_neurones_value = [256, 120, 84, 10, 1]
op_value = ['avg',"max"]

# Generation architectures

In [12]:
def stop(size_archi):
    prob = 5*size_archi
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [13]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    archi_file = open(directory+file_name, "a") # Open file in writting (a --> append)
    archi_file.write("""[
    """)
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

# Creation architecture without verification

In [16]:
for i in range (1,11):
    architecture = list()
    while(stop(len(architecture))):
        layer = layers_name[random.randrange(5)]
        # if convolution
        if(layer == Convolution):
            architecture.append(Convolution(
                kernel=kernel_value[random.randrange(5)], 
                padding=padding_value[random.randrange(2)], 
                stride=stride_value[random.randrange(5)], 
                nb_filter=nb_filter_value[random.randrange(8)],
                fct_activation=fct_activation_value[random.randrange(4)]
            ))
        # if input
        elif (layer == InputLayer):
            architecture.append(InputLayer(
                shape=[28,28,1]
            ))
        elif (layer == Pooling):
            architecture.append(Pooling(
                op = op_value[random.randrange(2)],
                kernel=kernel_value[random.randrange(5)], 
                padding=padding_value[random.randrange(2)], 
                stride=stride_value[random.randrange(5)]
            ))
        elif (layer == Dense):
            architecture.append(Dense(
                nb_neurones = nb_neurones_value[random.randrange(5)],
                fct_activation=fct_activation_value[random.randrange(4)]            
            ))
        elif (layer == Flatten) :
            architecture.append(Flatten())
        else : print("not possible")
    print(architecture)
    file_name = "archi_random_%d.json" % i
    print(file_name)
    print("")
    create_json_file(architecture, file_name)

[Flatten(), InputLayer(shape=[28, 28, 1]), Convolution(fct_activation='tanh', kernel=1, nb_filter=2048, padding='same', stride=1), Dense(fct_activation='relu', nb_neurones=10), Pooling(kernel=5, op='max', padding='valid', stride=2), Flatten(), Flatten()]
archi_random_1.json

[Dense(fct_activation='selu', nb_neurones=120), Pooling(kernel=3, op='avg', padding='valid', stride=1), InputLayer(shape=[28, 28, 1]), InputLayer(shape=[28, 28, 1]), Flatten(), Dense(fct_activation='relu', nb_neurones=84)]
archi_random_2.json

[Convolution(fct_activation='sigmoid', kernel=1, nb_filter=2048, padding='valid', stride=5), InputLayer(shape=[28, 28, 1]), Flatten(), Pooling(kernel=2, op='max', padding='same', stride=2), Flatten(), Convolution(fct_activation='relu', kernel=3, nb_filter=256, padding='valid', stride=4), Convolution(fct_activation='tanh', kernel=1, nb_filter=512, padding='valid', stride=4), Convolution(fct_activation='sigmoid', kernel=2, nb_filter=512, padding='same', stride=2)]
archi_random_

# Creation architecture with verification

In [ ]:
def verification_order(archi, layer):
    